<a href="https://colab.research.google.com/github/thomasjuin1/EmotionClassification/blob/twitter-dataset/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from datasets import load_dataset
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Load the dataset
dataset = load_dataset("dair-ai/emotion", "split")

# Split into train, validation, and test sets
train_set = dataset["train"]
val_set = dataset["validation"]
test_set = dataset["test"]

train_sset = train_set.map(lambda x: {"text": x["text"], "labels": [x["label"]]})
val_sset = val_set.map(lambda x: {"text": x["text"], "labels": [x["label"]]})
test_sset = test_set.map(lambda x: {"text": x["text"], "labels": [x["label"]]})

__emotions = [
    "sadness",
    "joy",
    "love",
    "anger",
    "fear",
    "surprise",
]

EMOTION_TO_INDEX = {
    emotion: index for index, emotion in enumerate(__emotions)
}

INDEX_TO_EMOTION = {
    index: emotion for emotion, index in EMOTION_TO_INDEX.items()
}